In [10]:
%load_ext autoreload
%autoreload 2
# from notion import get_notion_db_entries
# from calendar_api import delete_event, get_events, add_event
%run notion.py
%run calendar_api.py
%run utils.py

from datetime import datetime, timedelta, timezone
from utils import paris_tz
import json

update_threshold = 5  # days


# # Get all the dataframe with the events information
# df_events = notion_db_entries_to_event_df()

# # Keep only the events that have a date
# df_events = df_events[(df_events["start_date"] != "") & (
#     df_events["end_date"] != "") & (df_events["summary"] != "")]

# # Get last edited date else say it's 5 days before now
# # try:
# #     with open('meta.json', 'r') as meta_file:
# #         data = json.load(meta_file)
# #         last_updated_time = datetime.fromisoformat(
# #             data["last_updated_time"])
# # except Exception as error:
# #     print("An error occured while reading the meta file", error)
# #     last_updated_time = (datetime.now(paris_tz) -
# #                          timedelta(days=5)).isoformat()


# update_start_date = (datetime.now(paris_tz) -
#                      timedelta(days=update_threshold)).isoformat()

# # Keep only the events with edited time higher that last updated date

notion_db_entries_to_event_df().sort_values(by="start_date", ascending=False).head(10)

ImportError: cannot import name 'avoid_none' from 'utils' (/Users/jules/Documents/Jules/Code/notion/utils.py)

In [ ]:
df_events = df_events[df_events["start_date"] > update_start_date]
df_events.shape

In [ ]:
# Get all the events from two days before and for the next days
df_prev_events = get_events(update_start_date)
df_prev_events = df_prev_events.sort_values(by=["start_date"], ascending=True)
df_prev_events

In [ ]:
df_events = df_events.sort_values(by=["start_date"], ascending=True)
df_events

In [ ]:
# Create 4 cats : New, updated, unchanged, deleted
df_new_events = df_events[~df_events["id"].isin(df_prev_events["id"])]


In [ ]:
df_merged = pd.merge(df_events, df_prev_events, on='id', suffixes=('_current', '_prev'))
df_updated_events = df_merged[
    (df_merged['summary_current'] != df_merged['summary_prev']) |
    (df_merged['start_date_current'] != df_merged['start_date_prev']) |
    (df_merged['end_date_current'] != df_merged['end_date_prev']) |
    (df_merged['description_current'] != df_merged['description_prev'])
]
df_updated_events

In [ ]:

df_deleted_events = df_prev_events[~df_prev_events["id"].isin(df_events["id"])]

In [ ]:
df_new_events

In [ ]:
# New : create the event
df_new_events
for index, row in df_new_events.iterrows():
    add_event(row["id"], row["summary"], row["start_date"],
              row["end_date"], description=row["description"])
# Update get the event and update it
# for index, row in df_updated_events.iterrows():
#     delete_event(row["id"])
#     add_event(row["id"], row["summary"], row["start_date"],
#               row["end_date"], description=row["description"])

for index, row in df_deleted_events.iterrows():
    delete_event(row["id"])
    
    
# update the meta file
# last_updated_time = {
#     "last_updated_time": datetime.now(paris_tz).isoformat()}
# with open("meta.json", "w") as meta:
#     json.dump(last_updated_time, meta)